In [1192]:
import pandas as pd
import numpy as np
import decimal as D

In [1193]:
df = pd.read_csv("VSS910.csv", sep=";", parse_dates=["ReportDate"], dayfirst=True, thousands=".", decimal=",", usecols=["ReportDate", "BusinessTransactionCycle", "GrossAmount", "DefermentDays"], dtype={"GrossAmount": np.float64})

In [1194]:
df.DefermentDays.unique()

array([nan, 27.,  2.])

In [1195]:
df.insert(4, "eventType", "")

In [1196]:
df

,ReportDate,BusinessTransactionCycle,GrossAmount,DefermentDays,eventType
0,2021-01-01,Originals,9.216884e+04,NaN,
1,2021-01-01,Originals,1.782063e+06,27.0,
2,2021-01-01,Dispute Financial,2.856787e+05,NaN,
3,2021-01-01,Dispute Response Financial,3.203815e+04,NaN,
4,2021-01-01,Not Defined,6.207585e+04,NaN,
...,...,...,...,...,...
186,2021-01-24,Dispute Financial,2.151728e+05,NaN,
187,2021-01-24,Dispute Response Financial,5.322345e+04,NaN,
188,2021-01-24,Not Defined,6.315080e+03,NaN,
189,2021-01-24,Originals,8.670681e+07,NaN,


In [1197]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Originals" and row["DefermentDays"] == 27:  
        df.loc[index, "eventType"] = "credit"
    elif row["BusinessTransactionCycle"] == "Originals" and np.isnan(row["DefermentDays"]):
        df.loc[index, "eventType"] = "debit"

In [1198]:
for index, row in df.iterrows():
    if row["BusinessTransactionCycle"] == "Dispute Financial":
        df.loc[index, "eventType"] = "chargeback"
    elif row["BusinessTransactionCycle"] == "Dispute Response Financial":
        df.loc[index, "eventType"] = "secondPresentment"

In [1199]:
df["ReportDate"] = pd.to_datetime(df["ReportDate"])

In [1200]:
df["BusinessTransactionCycle"] = df["BusinessTransactionCycle"].astype("str")

In [1201]:
df.dtypes

ReportDate                  datetime64[ns]
BusinessTransactionCycle            object
GrossAmount                        float64
DefermentDays                      float64
eventType                           object
dtype: object

In [1202]:
df.head()

,ReportDate,BusinessTransactionCycle,GrossAmount,DefermentDays,eventType
0,2021-01-01,Originals,92168.84,NaN,debit
1,2021-01-01,Originals,1782062.81,27.0,credit
2,2021-01-01,Dispute Financial,285678.70,NaN,chargeback
3,2021-01-01,Dispute Response Financial,32038.15,NaN,secondPresentment
4,2021-01-01,Not Defined,62075.85,NaN,


In [1203]:
df = df.drop(columns=["BusinessTransactionCycle","DefermentDays"])

In [1204]:
df.ReportDate.unique()

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1205]:
df = df[['ReportDate', 'eventType', 'GrossAmount']]

In [1206]:
df

,ReportDate,eventType,GrossAmount
0,2021-01-01,debit,9.216884e+04
1,2021-01-01,credit,1.782063e+06
2,2021-01-01,chargeback,2.856787e+05
3,2021-01-01,secondPresentment,3.203815e+04
4,2021-01-01,,6.207585e+04
...,...,...,...
186,2021-01-24,chargeback,2.151728e+05
187,2021-01-24,secondPresentment,5.322345e+04
188,2021-01-24,,6.315080e+03
189,2021-01-24,debit,8.670681e+07


In [1207]:
df.rename(columns={"ReportDate": "confirmedDate",
                  "GrossAmount": "amountInBRL"}, inplace=True)

In [1208]:
#df.drop(df[df['eventType']=""].index, inplace = True) 

In [1209]:
rows_to_delete = []

In [1210]:
for index, row in df.iterrows():
    if row["eventType"] == "":
        rows_to_delete.append(index)

In [1211]:
rows_to_delete

[4,
 11,
 18,
 24,
 27,
 28,
 29,
 34,
 41,
 49,
 56,
 59,
 60,
 65,
 73,
 80,
 83,
 88,
 91,
 92,
 93,
 97,
 100,
 101,
 102,
 107,
 110,
 115,
 118,
 119,
 120,
 125,
 132,
 139,
 142,
 143,
 148,
 151,
 155,
 158,
 163,
 166,
 167,
 168,
 173,
 181,
 188]

In [1212]:
df = df.drop(rows_to_delete)

In [1213]:
dfCamila = df # (Output final)

In [1214]:
dfCamila
# ['confirmedDate', 'eventType', 'amountInBRL']) 

,confirmedDate,eventType,amountInBRL
0,2021-01-01,debit,9.216884e+04
1,2021-01-01,credit,1.782063e+06
2,2021-01-01,chargeback,2.856787e+05
3,2021-01-01,secondPresentment,3.203815e+04
5,2021-01-01,debit,9.012414e+07
...,...,...,...
185,2021-01-24,credit,1.632853e+06
186,2021-01-24,chargeback,2.151728e+05
187,2021-01-24,secondPresentment,5.322345e+04
189,2021-01-24,debit,8.670681e+07


In [1215]:
mov = pd.read_csv("MovementByConfirmedDate.csv", sep=';', parse_dates = ["ConfirmedDate"], dayfirst=True, thousands=",", dtype={"GrossAmount": np.float64, "NetAmount": np.float64}, usecols=["ConfirmedDate", "ServiceType", "GrossAmount"])

In [1216]:
mov.rename(columns={'ConfirmedDate':'confirmedDate',
                    'ServiceType':'eventType',
                    'GrossAmount':'amountInBRL'},
           inplace=True)

In [1217]:
# Teste
# Teste
mov.confirmedDate.unique()

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1218]:
for index, row in mov.iterrows():
    if row["eventType"] in ["Chargeback Suspension", "SecondPresentment Release", "Credit"]:
        mov.loc[index, "eventType"] = "credit"
    elif row["eventType"] == "SecondPresentment":
        mov.loc[index, "eventType"] = "secondPresentment"
    elif row["eventType"] == "Debit":
        mov.loc[index, "eventType"] = "debit"
    elif row["eventType"] == "Chargeback":
        mov.loc[index, "eventType"] = "chargeback"

In [1219]:
rows_to_delete = []

In [1220]:
for index, row in mov.iterrows():
    if row["eventType"] not in ['chargeback', 'credit', 'debit', 'secondPresentment']:
        rows_to_delete.append(index)

In [1221]:
rows_to_delete

[8,
 9,
 42,
 43,
 75,
 76,
 103,
 104,
 114,
 120,
 139,
 140,
 149,
 155,
 166,
 181,
 187,
 192,
 198,
 199,
 224,
 225,
 231,
 236,
 242,
 243,
 268,
 269,
 279,
 280,
 286,
 312,
 318,
 323,
 329,
 330,
 355,
 356,
 362,
 367,
 373,
 374,
 400,
 401,
 411,
 412,
 418,
 439,
 440,
 445,
 449,
 453,
 454,
 464,
 478,
 479,
 485,
 511,
 512,
 518,
 523,
 529,
 550,
 551,
 552,
 562,
 563,
 569,
 593,
 594,
 604,
 610,
 622,
 623,
 636,
 637,
 647,
 653,
 654,
 666,
 667,
 682,
 683,
 693,
 700,
 724,
 725,
 730,
 738,
 739,
 765,
 766,
 771,
 796,
 797,
 807,
 812,
 835,
 836,
 837,
 842,
 847,
 853,
 877,
 878,
 888,
 894,
 905,
 919,
 920,
 926,
 931,
 937,
 962,
 963,
 973,
 979,
 1004,
 1005,
 1010,
 1018,
 1042,
 1043,
 1048]

In [1222]:
mov = mov.drop(rows_to_delete)

In [1223]:
dfGustavo = mov # (Output final)

In [1224]:
vss = dfCamila
receivables = dfGustavo

In [1225]:
# Check

vss

,confirmedDate,eventType,amountInBRL
0,2021-01-01,debit,9.216884e+04
1,2021-01-01,credit,1.782063e+06
2,2021-01-01,chargeback,2.856787e+05
3,2021-01-01,secondPresentment,3.203815e+04
5,2021-01-01,debit,9.012414e+07
...,...,...,...
185,2021-01-24,credit,1.632853e+06
186,2021-01-24,chargeback,2.151728e+05
187,2021-01-24,secondPresentment,5.322345e+04
189,2021-01-24,debit,8.670681e+07


In [1226]:
# Convertendo datas dos dataframes para o formato YYYY-MM-DD






# PARAMOS AQUI
# Conseguimos usar o merge para juntar os dois dataframes (abaixo), mas vimos que as datas estão em formatos diferentes
# Precisamos converter ambas para o formato de data YYYY-MM-DD









In [1227]:
vss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 190
Data columns (total 3 columns):
confirmedDate    144 non-null datetime64[ns]
eventType        144 non-null object
amountInBRL      144 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.5+ KB


In [1228]:
receivables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 938 entries, 0 to 1061
Data columns (total 3 columns):
confirmedDate    938 non-null datetime64[ns]
eventType        938 non-null object
amountInBRL      938 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 29.3+ KB


In [1229]:
# Criação tabela de batimento

batimentoCleared = pd.DataFrame(columns = ['confirmedDate', 'eventType', 'vssAmount', 'receivablesAmount', 'dif']) 

###### Check tabelas

In [1230]:
vss

,confirmedDate,eventType,amountInBRL
0,2021-01-01,debit,9.216884e+04
1,2021-01-01,credit,1.782063e+06
2,2021-01-01,chargeback,2.856787e+05
3,2021-01-01,secondPresentment,3.203815e+04
5,2021-01-01,debit,9.012414e+07
...,...,...,...
185,2021-01-24,credit,1.632853e+06
186,2021-01-24,chargeback,2.151728e+05
187,2021-01-24,secondPresentment,5.322345e+04
189,2021-01-24,debit,8.670681e+07


In [1231]:
receivables

,confirmedDate,eventType,amountInBRL
0,2021-01-01,credit,-9.584689e+06
1,2021-01-01,credit,2.424347e+07
2,2021-01-01,credit,2.613886e+08
3,2021-01-01,credit,1.000000e+02
4,2021-01-01,credit,4.771980e+05
...,...,...,...
1057,2021-01-24,debit,7.109806e+06
1058,2021-01-24,debit,8.663760e+09
1059,2021-01-24,secondPresentment,5.242800e+04
1060,2021-01-24,secondPresentment,5.322345e+06


In [1232]:
batimentoCleared

,confirmedDate,eventType,vssAmount,receivablesAmount,dif


In [1233]:
debit = pd.DataFrame(columns = ['confirmedDate', 'dif']) 
credit = pd.DataFrame(columns = ['confirmedDate','dif']) 
chargeback = pd.DataFrame(columns = ['confirmedDate','dif']) 
secondPresentment = pd.DataFrame(columns = ['confirmedDate','dif']) 

###### Chech tabelas de batimento

In [1234]:
debit

,confirmedDate,dif


In [1235]:
credit

,confirmedDate,dif


In [1236]:
chargeback

,confirmedDate,dif


In [1237]:
secondPresentment

,confirmedDate,dif


In [1238]:
credit

,confirmedDate,dif


In [1239]:
# Datas de clearing distintas em cada base
vssDates = vss.confirmedDate.unique()
receivablesDates = receivables.confirmedDate.unique()


In [1240]:
vssDates

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1241]:
# Remoção de duplicatas entre base, composição da base única de datas distintas (allDates)
allDates = np.concatenate((vssDates,receivablesDates))
allUniqueDates = np.unique(allDates)

# Check
allUniqueDates

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1242]:
# Teste
receivables.confirmedDate.unique()

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [1243]:
debit

,confirmedDate,dif


In [1244]:
## Caso o batimento apresente divergências, uma hipótese a se investigar é o das datas estarem em dois formatos distintos
## Algumas em YYYY-MM-DD e outras em YYYY-DD-MM

In [1245]:
# Atribuindo datas distintas ao campo de data dos batimentos
credit.confirmedDate = allUniqueDates
debit.confirmedDate = allUniqueDates
chargeback.confirmedDate = allUniqueDates
secondPresentment.confirmedDate = allUniqueDates

In [1246]:
debit

,confirmedDate,dif
0,2021-01-01,NaN
1,2021-01-02,NaN
2,2021-01-03,NaN
3,2021-01-04,NaN
4,2021-01-05,NaN
5,2021-01-06,NaN
6,2021-01-07,NaN
7,2021-01-08,NaN
8,2021-01-09,NaN
9,2021-01-10,NaN


In [1247]:
# Incluindo amounts das bases originais nas bases de batimento


### Amounts Stone (Receivables)
creditAmounts_Stone = receivables[receivables.eventType == 'credit'].groupby("confirmedDate").sum()

debitAmounts_Stone = receivables[receivables.eventType == 'debit'].groupby("confirmedDate").sum()

chargebackAmounts_Stone = receivables[receivables.eventType == 'chargeback'].groupby("confirmedDate").sum()

secondPresentmentAmounts_Stone = receivables[receivables.eventType == 'secondPresentment'].groupby("confirmedDate").sum()


### Amounts Visa (VSS-910)
creditAmounts_Visa = vss[vss.eventType == 'credit'].groupby("confirmedDate").sum()

debitAmounts_Visa = vss[vss.eventType == 'debit'].groupby("confirmedDate").sum()

chargebackAmounts_Visa = vss[vss.eventType == 'chargeback'].groupby("confirmedDate").sum()

secondPresentmentAmounts_Visa = vss[vss.eventType == 'secondPresentment'].groupby("confirmedDate").sum()

In [1248]:
#Teste
chargebackAmounts_Visa

,amountInBRL
confirmedDate,
2021-01-01,285678.70
2021-01-02,90823.40
2021-01-03,108589.07
2021-01-04,77479.90
2021-01-05,456820.43
2021-01-06,234307.91
2021-01-07,295645.61
2021-01-08,422453.98
2021-01-09,562371.92


In [1249]:
#Incluindo amounts nas bases de batimento
credit = credit.merge(creditAmounts_Stone, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_banana','_Stone1'))
debit = debit.merge(debitAmounts_Stone, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_x','_y'))
chargeback = chargeback.merge(chargebackAmounts_Stone, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_x','_y'))
secondPresentment = secondPresentment.merge(secondPresentmentAmounts_Stone, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_x','_y'))

In [1250]:
creditAmounts_Visa

,amountInBRL
confirmedDate,
2021-01-01,1.051997e+08
2021-01-02,6.181466e+07
2021-01-03,9.060303e+07
2021-01-04,7.606153e+07
2021-01-05,8.943712e+07
2021-01-06,1.085501e+08
2021-01-07,1.043451e+08
2021-01-08,1.141037e+08
2021-01-09,1.181494e+08


In [1251]:
# Check
creditAmounts_Stone

,amountInBRL
confirmedDate,
2021-01-01,4.448963e+09
2021-01-02,9.338465e+09
2021-01-03,1.036869e+10
2021-01-04,1.410385e+10
2021-01-05,1.220003e+10
2021-01-06,1.296994e+10
2021-01-07,1.301408e+10
2021-01-08,1.607961e+10
2021-01-09,3.245436e+09


In [1252]:
credit

,confirmedDate,dif,amountInBRL
0,2021-01-01,NaN,4.448963e+09
1,2021-01-02,NaN,9.338465e+09
2,2021-01-03,NaN,1.036869e+10
3,2021-01-04,NaN,1.410385e+10
4,2021-01-05,NaN,1.220003e+10
5,2021-01-06,NaN,1.296994e+10
6,2021-01-07,NaN,1.301408e+10
7,2021-01-08,NaN,1.607961e+10
8,2021-01-09,NaN,3.245436e+09
9,2021-01-10,NaN,1.729702e+09


In [1253]:
# Teste
# credit
credit

,confirmedDate,dif,amountInBRL
0,2021-01-01,NaN,4.448963e+09
1,2021-01-02,NaN,9.338465e+09
2,2021-01-03,NaN,1.036869e+10
3,2021-01-04,NaN,1.410385e+10
4,2021-01-05,NaN,1.220003e+10
5,2021-01-06,NaN,1.296994e+10
6,2021-01-07,NaN,1.301408e+10
7,2021-01-08,NaN,1.607961e+10
8,2021-01-09,NaN,3.245436e+09
9,2021-01-10,NaN,1.729702e+09


In [57]:
# Merge das bases vss e receivables

credit = credit.merge(creditAmounts_Visa, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_Stone','_Visa'))
debit = debit.merge(debitAmounts_Visa, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_Stone','_Visa'))
chargeback = chargeback.merge(chargebackAmounts_Visa, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_Stone','_Visa'))
secondPresentment = secondPresentment.merge(secondPresentmentAmounts_Visa, how='left', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_Stone','_Visa'))

AttributeError: 'numpy.ndarray' object has no attribute 'get'

In [345]:
credit

,confirmedDate,amount_Stone,amount_Visa,dif
0,2021-02-05,[],NaN,NaN
1,2021-02-06,[],NaN,NaN
2,2021-01-01,[4448962506.0],NaN,NaN
3,2021-02-01,[9338465328.0],NaN,NaN
4,2021-03-01,[10368687977.0],NaN,NaN
5,2021-04-01,[14103846405.0],NaN,NaN
6,2021-05-01,[12200029905.0],NaN,NaN
7,2021-06-01,[12969942778.0],NaN,NaN
8,2021-07-01,[13014076504.0],NaN,NaN
9,2021-08-01,[16079613888.0],NaN,NaN


In [1255]:
# Teste
credit

,confirmedDate,dif,amountInBRL_Stone,amountInBRL_Visa
0,2021-01-01,NaN,4.448963e+09,1.051997e+08
1,2021-01-02,NaN,9.338465e+09,6.181466e+07
2,2021-01-03,NaN,1.036869e+10,9.060303e+07
3,2021-01-04,NaN,1.410385e+10,7.606153e+07
4,2021-01-05,NaN,1.220003e+10,8.943712e+07
5,2021-01-06,NaN,1.296994e+10,1.085501e+08
6,2021-01-07,NaN,1.301408e+10,1.043451e+08
7,2021-01-08,NaN,1.607961e+10,1.141037e+08
8,2021-01-09,NaN,3.245436e+09,1.181494e+08
9,2021-01-10,NaN,1.729702e+09,1.188169e+08


In [1256]:
# Merge das bases 
#credit = credit.merge(AllUniqueDates, how='inner', left_on='confirmedDate', right_on='confirmedDate', suffixes=('_Stone','_Visa'))
#debit = 
#chargeback =
#secondPresentment


#AllUniqueDates

In [1257]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 4 columns):
confirmedDate        24 non-null datetime64[ns]
dif                  0 non-null object
amountInBRL_Stone    24 non-null float64
amountInBRL_Visa     24 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 960.0+ bytes


Hora da Verdade XVIII

In [1258]:

# AQUI DEU BOM
# O batimento tá pronto. Já temos os 4 dataframes (credit,debit,chargeback,secondPresentment) preenchidos com valores e dif
# Só falta converter os valores para R$ e HOMOLOGAR





credit 

,confirmedDate,dif,amountInBRL_Stone,amountInBRL_Visa
0,2021-01-01,NaN,4.448963e+09,1.051997e+08
1,2021-01-02,NaN,9.338465e+09,6.181466e+07
2,2021-01-03,NaN,1.036869e+10,9.060303e+07
3,2021-01-04,NaN,1.410385e+10,7.606153e+07
4,2021-01-05,NaN,1.220003e+10,8.943712e+07
5,2021-01-06,NaN,1.296994e+10,1.085501e+08
6,2021-01-07,NaN,1.301408e+10,1.043451e+08
7,2021-01-08,NaN,1.607961e+10,1.141037e+08
8,2021-01-09,NaN,3.245436e+09,1.181494e+08
9,2021-01-10,NaN,1.729702e+09,1.188169e+08


In [1260]:
credit.dif = credit.amountInBRL_Stone - credit.amountInBRL_Visa

In [1261]:
credit

,confirmedDate,dif,amountInBRL_Stone,amountInBRL_Visa
0,2021-01-01,4.343763e+09,4.448963e+09,1.051997e+08
1,2021-01-02,9.276651e+09,9.338465e+09,6.181466e+07
2,2021-01-03,1.027808e+10,1.036869e+10,9.060303e+07
3,2021-01-04,1.402778e+10,1.410385e+10,7.606153e+07
4,2021-01-05,1.211059e+10,1.220003e+10,8.943712e+07
5,2021-01-06,1.286139e+10,1.296994e+10,1.085501e+08
6,2021-01-07,1.290973e+10,1.301408e+10,1.043451e+08
7,2021-01-08,1.596551e+10,1.607961e+10,1.141037e+08
8,2021-01-09,3.127287e+09,3.245436e+09,1.181494e+08
9,2021-01-10,1.610885e+09,1.729702e+09,1.188169e+08


Check dos dataframes

In [325]:
creditAmounts_Visa

,amountInBRL
confirmedDate,
2021-02-05,1.161185e+08
2021-02-06,1.190148e+08


In [326]:
credit

,confirmedDate,dif,amountInBRL_Stone,amountInBRL_Visa
0,2021-02-05,NaN,NaN,1.161185e+08
1,2021-02-06,NaN,NaN,1.190148e+08
2,2021-01-01,NaN,4.448963e+09,NaN
3,2021-01-02,NaN,9.338465e+09,NaN
4,2021-01-03,NaN,1.036869e+10,NaN
5,2021-01-04,NaN,1.410385e+10,NaN
6,2021-01-05,NaN,1.220003e+10,NaN
7,2021-01-06,NaN,1.296994e+10,NaN
8,2021-01-07,NaN,1.301408e+10,NaN
9,2021-01-08,NaN,1.607961e+10,NaN


In [327]:
debit

,confirmedDate,dif,amountInBRL_Stone,amountInBRL_Visa
0,2021-02-05,NaN,NaN,8.411496e+07
1,2021-02-06,NaN,NaN,1.064716e+08
2,2021-01-01,NaN,1.301754e+10,NaN
3,2021-01-02,NaN,1.008725e+10,NaN
4,2021-01-03,NaN,1.108273e+10,NaN
5,2021-01-04,NaN,4.642707e+09,NaN
6,2021-01-05,NaN,9.718785e+09,NaN
7,2021-01-06,NaN,1.793537e+10,NaN
8,2021-01-07,NaN,2.010755e+10,NaN
9,2021-01-08,NaN,2.154116e+10,NaN


In [328]:
# Transformar o confirmedDate dos dataframes parciais de um índice para uma coluna

creditAmounts_Stone.reset_index(inplace=True)
creditAmounts_Stone = creditAmounts_Stone.rename(columns = {'index':'confirmedDate'})

debitAmounts_Stone.reset_index(inplace=True)
debitAmounts_Stone = debitAmounts_Stone.rename(columns = {'index':'confirmedDate'})

chargebackAmounts_Stone.reset_index(inplace=True)
chargebackAmounts_Stone = chargebackAmounts_Stone.rename(columns = {'index':'confirmedDate'})

secondPresentmentAmounts_Stone.reset_index(inplace=True)
secondPresentmentAmounts_Stone = secondPresentmentAmounts_Stone.rename(columns = {'index':'confirmedDate'})

creditAmounts_Visa.reset_index(inplace=True)
creditAmounts_Visa = creditAmounts_Visa.rename(columns = {'index':'confirmedDate'})

debitAmounts_Visa.reset_index(inplace=True)
debitAmounts_Visa = debitAmounts_Visa.rename(columns = {'index':'confirmedDate'})

chargebackAmounts_Visa.reset_index(inplace=True)
chargebackAmounts_Visa = chargebackAmounts_Visa.rename(columns = {'index':'confirmedDate'})

secondPresentmentAmounts_Visa.reset_index(inplace=True)
secondPresentmentAmounts_Visa = secondPresentmentAmounts_Visa.rename(columns = {'index':'confirmedDate'})

Check dataframes

In [329]:
for date in allDates:
    #Incluindo amounts Stone nas bases de batimento
    credit.iat[credit[credit.confirmedDate == date].index.values.astype(int)[0], 1] = creditAmounts_Stone[creditAmounts_Stone.confirmedDate == date].amountInBRL
    debit[debit.confirmedDate == date].amount_Stone = debitAmounts_Stone[debitAmounts_Stone.confirmedDate == date].amountInBRL
    chargeback[chargeback.confirmedDate == date].amount_Stone = chargebackAmounts_Stone[chargebackAmounts_Stone.confirmedDate == date].amountInBRL
    secondPresentment[secondPresentment.confirmedDate == date].amount_Stone = secondPresentmentAmounts_Stone[secondPresentmentAmounts_Stone.confirmedDate == date].amountInBRL
    
    #Incluindo amounts Visa nas bases de batimento
    credit[credit.confirmedDate == date].amount_Visa = creditAmounts_Visa[creditAmounts_Visa.confirmedDate == date].amountInBRL
    debit[debit.confirmedDate == date].amount_Visa = debitAmounts_Visa[debitAmounts_Visa.confirmedDate == date].amountInBRL
    chargeback[chargeback.confirmedDate == date].amount_Visa = chargebackAmounts_Visa[chargebackAmounts_Visa.confirmedDate == date].amountInBRL
    secondPresentment[secondPresentment.confirmedDate == date].amount_Visa = secondPresentmentAmounts_Visa[secondPresentmentAmounts_Visa.confirmedDate == date].amountInBRL
    
    
    
    # Fazer batimento em cada data
    credit[credit.confirmedDate == date].dif = credit[credit.confirmedDate == date].amount_Visa - credit[credit.confirmedDate == date].amount_Stone
    debit[debit.confirmedDate == date].dif = debit[debit.confirmedDate == date].amount_Visa - debit[debit.confirmedDate == date].amount_Stone
    chargeback[chargeback.confirmedDate == date].dif = chargeback[chargeback.confirmedDate == date].amount_Visa - chargeback[chargeback.confirmedDate == date].amount_Stone
    secondPresentment[secondPresentment.confirmedDate == date].dif = secondPresentment[secondPresentment.confirmedDate == date].amount_Visa - secondPresentment[secondPresentment.confirmedDate == date].amount_Stone
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if __name__ == '__main__'

AttributeError: 'DataFrame' object has no attribute 'amount_Visa'

In [ ]:
## Mais uma hipótese de erro é que tenhamos considerado valores em USD como BRL, porque não tratamos o campo de currency
## das bases originais

Hora da Verdade

In [196]:
debit

,confirmedDate,dif


In [ ]:
credit.amount_Stone

In [ ]:
credit

In [ ]:
chargeback

In [ ]:
secondPresentment